In [2]:
import pandas as pd

In [65]:

with open('revenue_quarterly.parquet', 'rb') as f:
    df = pd.read_parquet(f)
print(df.head(2))
print(len(df.ticker.unique()))
print(len(df))


                      accn      cik                  entityName    loc  \
1490  0001090872-18-000015  1090872  Agilent Technologies, Inc.  US-CA   
1081  0001090872-19-000006  1090872  Agilent Technologies, Inc.  US-CA   

           start         end           val ticker  \
1490  2017-05-01  2017-07-31  1.114000e+09      a   
1081  2017-11-01  2018-01-31  1.211000e+09      a   

                                                    tag       ccp  uom  \
1490  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q2  USD   
1081  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q4  USD   

      tag_rank  
1490         1  
1081         1  
5498
134916


In [66]:
import re
_MONTH_BY_QUARTER = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}

df['year'] = df["ccp"].apply(lambda x: int(re.search('CY(\d+)Q', x).group(1)))
df["quarter"] = df["ccp"].apply(
    lambda x: re.search("CY\d+(Q\d)", x).group(1)
)
df.head(2)

,accn,cik,entityName,loc,start,end,val,ticker,tag,ccp,uom,tag_rank,year,quarter
1490,0001090872-18-000015,1090872,"Agilent Technologies, Inc.",US-CA,2017-05-01,2017-07-31,1.114000e+09,a,RevenueFromContractWithCustomerExcludingAssess...,CY2017Q2,USD,1,2017,Q2
1081,0001090872-19-000006,1090872,"Agilent Technologies, Inc.",US-CA,2017-11-01,2018-01-31,1.211000e+09,a,RevenueFromContractWithCustomerExcludingAssess...,CY2017Q4,USD,1,2017,Q4


In [67]:
import datetime
print(df.head(2))
df["datetime"] = df.apply(
    lambda x: datetime.datetime(x.year, _MONTH_BY_QUARTER[x.quarter], 1), axis=1
)
df = df.set_index('datetime')
df.head(2)

                      accn      cik                  entityName    loc  \
1490  0001090872-18-000015  1090872  Agilent Technologies, Inc.  US-CA   
1081  0001090872-19-000006  1090872  Agilent Technologies, Inc.  US-CA   

           start         end           val ticker  \
1490  2017-05-01  2017-07-31  1.114000e+09      a   
1081  2017-11-01  2018-01-31  1.211000e+09      a   

                                                    tag       ccp  uom  \
1490  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q2  USD   
1081  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q4  USD   

      tag_rank  year quarter  
1490         1  2017      Q2  
1081         1  2017      Q4  


,accn,cik,entityName,loc,start,end,val,ticker,tag,ccp,uom,tag_rank,year,quarter
datetime,,,,,,,,,,,,,,
2017-06-01,0001090872-18-000015,1090872,"Agilent Technologies, Inc.",US-CA,2017-05-01,2017-07-31,1.114000e+09,a,RevenueFromContractWithCustomerExcludingAssess...,CY2017Q2,USD,1,2017,Q2
2017-12-01,0001090872-19-000006,1090872,"Agilent Technologies, Inc.",US-CA,2017-11-01,2018-01-31,1.211000e+09,a,RevenueFromContractWithCustomerExcludingAssess...,CY2017Q4,USD,1,2017,Q4


In [91]:
def reindex_ticker(group):
    min_q = group.index.min()
    max_q = group.index.max()
    group = group.reindex(pd.date_range(min_q, max_q, freq='3MS'))
    group['val'] = group['val'].interpolate()
    group['ticker'] = group['ticker'].ffill()
    return group.reset_index(names='datetime')[['datetime', 'ticker', 'val']]
print(df.head(2))
df.groupby('ticker').apply(reindex_ticker).reset_index(drop=True)

                            accn      cik                  entityName    loc  \
datetime                                                                       
2017-06-01  0001090872-18-000015  1090872  Agilent Technologies, Inc.  US-CA   
2017-12-01  0001090872-19-000006  1090872  Agilent Technologies, Inc.  US-CA   

                 start         end           val ticker  \
datetime                                                  
2017-06-01  2017-05-01  2017-07-31  1.114000e+09      a   
2017-12-01  2017-11-01  2018-01-31  1.211000e+09      a   

                                                          tag       ccp  uom  \
datetime                                                                       
2017-06-01  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q2  USD   
2017-12-01  RevenueFromContractWithCustomerExcludingAssess...  CY2017Q4  USD   

            tag_rank  year quarter  
datetime                            
2017-06-01         1  2017      Q2  
2017-12-01

/tmp/ipykernel_103397/2573015747.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('ticker').apply(reindex_ticker).reset_index(drop=True)


,datetime,ticker,val
0,2017-06-01,a,1.114000e+09
1,2017-09-01,a,1.162500e+09
2,2017-12-01,a,1.211000e+09
3,2018-03-01,a,1.206000e+09
4,2018-06-01,a,1.203000e+09
...,...,...,...
168980,2021-09-01,zzll,3.422280e+07
168981,2021-12-01,zzll,1.731419e+07
168982,2022-03-01,zzll,4.055830e+05
168983,2022-06-01,zzll,2.533610e+05


In [90]:
def reindex_ticker(group):
    min_q = group.index.min()
    max_q = group.index.max()

    # 1. Resample with clearer method
    group = group.resample('3MS').asfreq() 

    # 2. Interpolate and forward-fill directly
    group['val'] = group['val'].interpolate()
    group['ticker'] = group['ticker'].ffill()

    # 3. Simplified output
    return group.reset_index('datetime')[['datetime', 'ticker', 'val']].reset_index() 

# ... (Assuming your 'df' is already defined)
result = df.groupby('ticker').apply(reindex_ticker)
print(result.head()) 

          index   datetime ticker           val
ticker                                         
a      0      0 2017-06-01      a  1.114000e+09
       1      1 2017-09-01      a  1.162500e+09
       2      2 2017-12-01      a  1.211000e+09
       3      3 2018-03-01      a  1.206000e+09
       4      4 2018-06-01      a  1.203000e+09


/tmp/ipykernel_103397/3198416721.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('ticker').apply(reindex_ticker)


In [69]:
pd.timeseries.offset_aliases

AttributeError: module 'pandas' has no attribute 'timeseries'

6601


,ticker,ccp,level_2,tag,val
